In [2]:
class PHC3salcomp:
    '''
    class SALcomp
             
    -use layerwise = True to compare a set of depth
    -use depth_limit to specify maximum depth for mean-over-depth comparison
    -Projection 'rob' (Robinson) results in strange values, please use only 'pc' Plate Carree!
    -Use of 'use_temp_mask' loads additional FESOM 'temp' out put to use as a mask for the 'salt' output.
    '''
    def __init__(self,resultpath,savepath,mesh,ncpath,first_year,last_year,
                 PHCvar='salt',
                 mapproj = 'rob',
                 savefig=False,
                 cmap = 'viridis',
                 get_overview = False,
                 use_temp_mask = False,
                 layerwise=False,
                 depth_array=(0,50,200,1000,2000,4000),
                 uplow=[0, 100],
                 cmap_extension='both',
                 verbose=True,
                 plotting=True,
                 output=False,
                 Taylor=True,
                 runname='fesom'):

        self.runname = runname
        self.resultpath = resultpath
        self.savepath = savepath
        self.mesh = mesh
        self.ncpath = ncpath
        self.fyear = first_year
        self.lyear = last_year
        self.PHCvar = PHCvar
        self.mapproj = mapproj
        self.cmap = cmap
        self.savefig = savefig
        self.get_overview = get_overview
        self.use_temp_mask = use_temp_mask
        self.layerwise = layerwise
        self.depth_array = depth_array
        self.uplow = uplow
        self.cmap_extension = cmap_extension
        self.verbose = verbose
        self.plotting = plotting
        self.Taylor = Taylor
        
        import matplotlib.pyplot as plt
        import numpy as np
        import skill_metrics as sm
        import cartopy.crs as ccrs
        import pyfesom2 as pf
        from load_interp_PHC import PHCdata
        from Py_f2recom_toolbox import plt_Taylor_norm
        from Py_f2recom_toolbox import plt_Taylor_comp

        if self.mapproj == 'rob':
            box=[-180, 180, -90, 90]
        elif self.mapproj == 'pc':
            box=[-180, 180, -90, 90]
        elif self.mapproj == 'sp':
            box=[-180, 180, -90, -30]
        elif self.mapproj == 'np':
            box=[-180, 180, 60, 90]
            
        self.mapproj = pf.get_proj(self.mapproj)   
        years = np.arange(first_year,last_year+1,1)

        labelfesom = 'FESOM {0}-{1}'.format(self.fyear,self.lyear)
        unitfesom = 'Salinity' 

        # load data -------------------------------------------------------------------------------------
        fesom = pf.get_data(resultpath, "salt", years, mesh, 
                               how="mean", compute=True, runid=self.runname, silent=self.verbose)

        if(use_temp_mask == True):
            if(self.verbose):
                print('use_temp_mask = True\nLoad temperature data for masking...')
            TEMPfesom = pf.get_data(resultpath, "temp", years, mesh, 
                               how="mean", compute=True, runid=self.runname, silent=self.verbose)
            fesom[TEMPfesom == 0] = 0 # apply mask

            
        # load PHC data  -------------------------------------------------------------------------------------
        PHC_input = PHCdata(self.runname,self.resultpath,self.mesh,self.ncpath,self.PHCvar, get_overview=self.get_overview)
        sal_int = PHC_input.phc_int    
        
        labelphc = 'PHC'
        unitphc = 'Salinity' 

        # apply sea mask to PHC as in FESOM ----------------------------------------------------------------------------------
        # assumption: there is no ocean where value in FESOM == 0
        sal_int_ma = np.copy(sal_int)
        sal_int_ma[fesom == 0] = 0

        # plot PHC and FESOM ----------------------------------------------------------------------------------
        if(self.layerwise):
            if(self.depth_array == []):
                depth_array = (0,50,200,1000,2000,4000)
                
            for i in np.arange(len(depth_array)-1):
                if depth_array[i+1] < -np.max(WOA_input.layer_depths):
                    print('{0}m is too deep for climatology.\nPlease consider choosing max depth at {1}!\n***'.format(uplow[1],np.max(PHC_input.layer_depths)))

                uplow = [depth_array[i], depth_array[i+1]]

                fesom_mean = pf.layermean_data(fesom, mesh, uplow = uplow)
                woa_int_ma_mean = pf.layermean_data(sal_int_ma, mesh, uplow = uplow)

                if(self.verbose):
                    print('\nFESOM mean = {0:4.4f}, std = {1:4.4f}, min = {2:4.4f}, max = {3:4.4f}\nWOA mean = {4:4.4f}, std = {5:4.4f}, min = {6:4.4f}, max = {7:4.4f}'.format(
                        np.nanmean(fesom_mean),np.nanstd(fesom_mean),np.nanmin(fesom_mean),np.nanmax(fesom_mean),
                        np.nanmean(sal_int_ma_mean),np.nanstd(sal_int_ma_mean),np.nanmin(sal_int_ma_mean),np.nanmax(sal_int_ma_mean)))


                if plotting:
                    fig = plt.figure(figsize=(15,12), constrained_layout=False)
                    axes = fig.subplot_mosaic(
                                """
                                AB
                                CC
                                """,
                                gridspec_kw={'hspace': 0.1, 'wspace': 0.1, 'bottom': 0.03}, 
                                subplot_kw=dict(projection=self.mapproj))

                    m1 = axes['A']
                    levels = np.arange(23,37,.5)
                    f1 = pf.subplot(mesh, fig, m1, [fesom[:,i]],
                                levels = levels,
                                units=unitphc, 
                                mapproj=self.mapproj, # robinson projection takes more time!
                                cmap = self.cmap,cmap_extension=self.cmap_extension,
                                titles=labelfesom +'\n ({0}-{1} m)'.format(uplow[0],uplow[1]),
                                box=box,
                               )

                    m2 = axes['B']
                    f2 = pf.subplot(mesh, fig, m2, [sal_int_ma[:,i]], 
                                levels = levels,
                                units=unitphc, 
                                mapproj=self.mapproj, # robinson projection takes more time!
                                cmap = self.cmap,cmap_extension=self.cmap_extension,
                                titles=labelphc +'\n ({0}-{1} m)'.format(uplow[0],uplow[1]),
                                box=box,
                               )

                    cbar1_ax = fig.add_axes([0.13, 0.53, 0.76, 0.02])
                    cbar1 = fig.colorbar(f1,
                                    cax = cbar1_ax, 
                                    orientation = 'horizontal',
                                    fraction=0.046, pad=0.04) 
                    cbar1.set_label(unitfesom, fontsize=18)
                    cbar1.ax.tick_params(labelsize=18)

                    m3 = axes['C']

                    levels_diff = np.arange(-5,5.25,0.25)
                    f3 = pf.subplot(mesh, fig, m3, [fesom[:,i]-sal_int_ma[:,i]], 
                                rowscol= (1,1),
                                levels = levels_diff,
                                units=unitphc, 
                                mapproj=self.mapproj, # robinson projection takes more time!
                                cmap = 'RdBu_r',
                                titles='FESOM - PHC\n ({0}-{1} m)'.format(uplow[0],uplow[1]),
                                box=box,
                               )

                    fig.subplots_adjust(bottom=0.16)
                    cbar2_ax = fig.add_axes([0.13, 0.001, 0.76, 0.02])
                    cbar2 = fig.colorbar(f3,
                                    cax = cbar2_ax, 
                                    orientation = 'horizontal',
                                    fraction=0.046, pad=0.04) 
                    cbar2.set_label(unitfesom, fontsize=18)
                    cbar2.ax.tick_params(labelsize=18)
                    
                    plt.show(block=False)
                    # fig export  -------------------------------------------------------------------------------------
                    if(self.savefig==True):
                        plt.savefig(self.savepath+self.runname+'_'+'Sal_PHC'+'_'+str(years[0])+'to'+str(years[-1])+'_{0}-{1}m.png'.format(uplow[0], uplow[1]), 
                                dpi = 300, bbox_inches='tight')
                        plt.savefig(self.savepath+self.runname+'_'+'Sal_PHC'+'_'+str(years[0])+'to'+str(years[-1])+'_{0}-{1}m.png'.format(uplow[0], uplow[1]), 
                                bbox_inches='tight')
                    
                          

            if Taylor:
                # statistics  -------------------------------------------------------------------------------------
                # preparation of datasets
                if np.isnan(np.min(sal_int_ma)): print('WARNING: The interpolated PHC field contains NaNs at depth')
                if np.isnan(np.min(fesom)): print('WARNING: The interpolated FESOM field contains NaNs at depth')       

                title = 'Normalized Taylor Diagram for Salinity'
                plt_Taylor_comp(sal_int_ma,SALfesom, mask=True, title=title, depth_array=depth_array, mesh=mesh,verbose = True)
                
                plt.show(block=False)
                # fig export  -------------------------------------------------------------------------------------
                if(self.savefig==True):                
                    plt.savefig(self.savepath+self.runname+'_'+'Sal_PHC_Taylor'+'_'+str(years[0])+'to'+str(years[-1])+'_{0}-{1}m.png'.format(uplow[0], uplow[1])),
                    plt.savefig(self.savepath+self.runname+'_'+'Sal_PHC_Taylor'+'_'+str(years[0])+'to'+str(years[-1])+'_{0}-{1}m.png'.format(uplow[0], uplow[1])),
                
                
            if output:
                    print('Only return non-layerwise output')
        
        # mean over depth  -------------------------------------------------------------------------------------
        else:
            if uplow[1] < -np.max(PHC_input.layer_depths):
                    print('{0}m is too deep for climatology.\nPlease consider choosing max depth at {1}!\n***'.format(uplow[1],np.max(GLODAP_input.layer_depths)))

            fesom_mean = pf.layermean_data(fesom, mesh, uplow = uplow)
            sal_int_ma_mean = pf.layermean_data(sal_int_ma, mesh, uplow = uplow)

            if(self.verbose):
                print('\nFESOM mean = {0:4.4f}, std = {1:4.4f}, min = {2:4.4f}, max = {3:4.4f}\nPHC mean = {4:4.4f}, std = {5:4.4f}, min = {6:4.4f}, max = {7:4.4f}'.format(
                np.nanmean(fesom_mean),np.nanstd(fesom_mean),np.nanmin(fesom_mean),np.nanmax(fesom_mean),
                np.nanmean(sal_int_ma_mean),np.nanstd(sal_int_ma_mean),np.nanmin(sal_int_ma_mean),np.nanmax(sal_int_ma_mean)))
            
            if plotting:
                fig = plt.figure(figsize=(15,12), constrained_layout=False)
                axes = fig.subplot_mosaic(
                            """
                            AB
                            CC
                            """,
                            gridspec_kw={'hspace': 0.1, 'wspace': 0.1, 'bottom': 0.03}, 
                            subplot_kw=dict(projection=self.mapproj))
                    
                m1 = axes['A']
                levels = np.arange(23,37,.5)
                f1 = pf.subplot(mesh, fig, m1, [fesom_mean],
                            levels = levels,
                            units=unitphc, 
                            mapproj=self.mapproj, # robinson projection takes more time!
                            cmap = self.cmap,cmap_extension=self.cmap_extension,
                            titles=labelfesom +'\n ({0}-{1} m)'.format(uplow[0],uplow[1]),
                            box=box,
                           )
                    
                m2 = axes['B']
                f2 = pf.subplot(mesh, fig, m2, [sal_int_ma_mean], 
                            levels = levels,
                            units=unitphc, 
                            mapproj=self.mapproj, # robinson projection takes more time!
                            cmap = self.cmap,cmap_extension=self.cmap_extension,
                            titles=labelphc +'\n ({0}-{1} m)'.format(uplow[0],uplow[1]),
                            box=box,
                           )
                    
                cbar1_ax = fig.add_axes([0.13, 0.53, 0.76, 0.02])
                cbar1 = fig.colorbar(f1,
                                cax = cbar1_ax, 
                                orientation = 'horizontal',
                                fraction=0.046, pad=0.04) 
                cbar1.set_label(unitfesom, fontsize=18)
                cbar1.ax.tick_params(labelsize=18)
        
                m3 = axes['C']

                levels_diff = np.arange(-5,5.25,0.25)
                f3 = pf.subplot(mesh, fig, m3, [fesom_mean-sal_int_ma_mean], 
                            rowscol= (1,1),
                            levels = levels_diff,
                            units=unitphc, 
                            mapproj=self.mapproj, # robinson projection takes more time!
                            cmap = 'RdBu_r',
                            titles='FESOM - PHC'+'\n ({0}-{1} m)'.format(uplow[0],uplow[1]),
                            box=box,
                           )
                
                fig.subplots_adjust(bottom=0.02)
                cbar2_ax = fig.add_axes([0.13, 0.001, 0.76, 0.02])
                cbar2 = fig.colorbar(f3,
                                cax = cbar2_ax, 
                                orientation = 'horizontal',
                                fraction=0.046, pad=0.04) 
                cbar2.set_label(unitfesom, fontsize=18)
                cbar2.ax.tick_params(labelsize=18)
                
                
                # fig export  -------------------------------------------------------------------------------------
                if(self.savefig==True):
                    plt.savefig(self.savepath+self.runname+'_'+'Sal_PHC'+'_'+str(years[0])+'to'+str(years[-1])+'_{0}-{1}m.png'.format(uplow[0], uplow[1]),
                                dpi = 300, bbox_inches='tight')
                    plt.savefig(self.savepath+self.runname+'_'+'Sal_PHC'+'_'+str(years[0])+'to'+str(years[-1])+'_{0}-{1}m.png'.format(uplow[0], uplow[1]),
                                bbox_inches='tight')
                plt.show(block=False)
                     
                
            if Taylor:
                # statistics  -------------------------------------------------------------------------------------            
                # preparation of datasets
                if np.isnan(np.min(sal_int_ma_mean)): print('WARNING: The interpolated PHC field contains NaNs at depth')
                if np.isnan(np.min(fesom_mean)): print('WARNING: The interpolated FESOM field contains NaNs at depth')

                title = 'Taylor Diagram for Salinity \n(mean over depth, max = {0}m)'.format(uplow[0],uplow[1]),
                plt_Taylor_norm(sal_int_ma_mean,fesom_mean,mask=True,title=title,verbose = True)
                
                
                # fig export  -------------------------------------------------------------------------------------
                if(self.savefig==True):                
                    plt.savefig(self.savepath+self.runname+'_'+'Sal_PHC_Taylor'+'_'+str(years[0])+'to'+str(years[-1])+'.png', dpi = 300, bbox_inches='tight')
                    plt.savefig(self.savepath+self.runname+'_'+'Sal_PHC_Taylor'+'_'+str(years[0])+'to'+str(years[-1])+'.pdf', bbox_inches='tight')
                plt.show(block=False)
                
        if output:
            self.sal_fesom = fesom_mean
            self.sal_phc   = sal_int_ma_mean

In [3]:
class PHC3tempcomp:
    '''
    class TEMPcomp
    
    Compare temperature data of FESOM-REcoM to PHC.
    
    -Specify depth_limit to use only upper layers with this depth maximum.
    -Use layerwise = True to compare a set of depth.
    '''
    def __init__(self,resultpath,savepath,mesh,ncpath,first_year,last_year,
                 PHCvar='temp',
                 mapproj='rob',
                 cmap = 'inferno',
                 savefig=False,
                 layerwise=False,
                 depth_array=(0,50,200,1000,2000,4000),
                 uplow=[0, 100],
                 cmap_extension='max',
                 verbose=True,
                 get_overview = False,
                 plotting=True,
                 output=False,
                 Taylor=True,
                 runname='fesom'):
        

        self.runname = runname
        self.resultpath = resultpath
        self.savepath = savepath
        self.mesh = mesh
        self.ncpath = ncpath
        self.fyear = first_year
        self.lyear = last_year
        self.PHCvar = PHCvar
        self.mapproj = mapproj
        self.cmap = cmap
        self.savefig = savefig
        self.layerwise = layerwise
        self.depth_array = depth_array
        self.uplow = uplow
        self.cmap_extension = cmap_extension
        self.verbose = verbose
        self.get_overview = get_overview
        self.plotting = plotting
        self.output = output
        self.Taylor = Taylor
        
        import matplotlib.pyplot as plt
        import numpy as np
        import skill_metrics as sm
        import cartopy.crs as ccrs
        import pyfesom2 as pf
        from load_interp_PHC import PHCdata
        from Py_f2recom_toolbox import plt_Taylor_norm
        from Py_f2recom_toolbox import plt_Taylor_comp
        
        if self.mapproj == 'rob':
            box=[-180, 180, -90, 90]
        elif self.mapproj == 'pc':
            box=[-180, 180, -90, 90]
        elif self.mapproj == 'sp':
            box=[-180, 180, -90, -30]
        elif self.mapproj == 'np':
            box=[-180, 180, 60, 90]
        
        self.mapproj = pf.get_proj(self.mapproj)     
        years = np.arange(first_year,last_year+1,1)

        labelfesom = 'FESOM {0}-{1}'.format(self.fyear,self.lyear)
        unitfesom = 'T [$^{\circ}$C]'       

        # load data -------------------------------------------------------------------------------------
        fesom = pf.get_data(resultpath, "temp", years, mesh, 
                               how="mean", compute=True, runid=self.runname, silent=self.verbose)

        # load PHC data  -------------------------------------------------------------------------------------
        PHC_input = PHCdata(self.runname,self.resultpath,self.mesh,self.ncpath,self.PHCvar, get_overview=self.get_overview)
        temp_int = PHC_input.phc_int    
        
        labelphc = 'PHC'
        unitphc = 'T [$^{\circ}$C]' 

        # apply sea mask to PHC as in FESOM ----------------------------------------------------------------------------------
        # assumption: there is no ocean where value in FESOM == 0
        temp_int_ma = np.copy(temp_int)
        temp_int_ma[fesom == 0] = 0

        # plot PHC and FESOM ----------------------------------------------------------------------------------
        if(self.layerwise):
            if(self.depth_array == []):
                depth_array = (0,50,200,1000,2000,4000)
                
            for i in np.arange(len(depth_array)-1):
                if depth_array[i+1] < -np.max(PHC_input.layer_depths):
                    print('{0}m is too deep for climatology.\nPlease consider choosing max depth at {1}!\n***'.format(uplow[1],np.max(PHC_input.layer_depths)))

                uplow = [depth_array[i], depth_array[i+1]]

                fesom_mean = pf.layermean_data(fesom, mesh, uplow = uplow)
                temp_int_ma_mean = pf.layermean_data(temp_int_ma, mesh, uplow = uplow)

                if(self.verbose):
                    print('\nFESOM mean = {0:4.4f}, std = {1:4.4f}, min = {2:4.4f}, max = {3:4.4f}\nPHC mean = {4:4.4f}, std = {5:4.4f}, min = {6:4.4f}, max = {7:4.4f}'.format(
                        np.nanmean(fesom_mean),np.nanstd(fesom_mean),np.nanmin(fesom_mean),np.nanmax(fesom_mean),
                        np.nanmean(temp_int_ma_mean),np.nanstd(temp_int_ma_mean),np.nanmin(temp_int_ma_mean),np.nanmax(temp_int_ma_mean)))

                if plotting:
                    fig = plt.figure(figsize=(15,12), constrained_layout=False)
                    axes = fig.subplot_mosaic(
                                """
                                AB
                                CC
                                """,
                                gridspec_kw={'hspace': 0.1, 'wspace': 0.1, 'bottom': 0.03}, 
                                subplot_kw=dict(projection=self.mapproj))

                    m1 = axes['A']
                    levels = np.arange(-2,25,1)
                    f1 = pf.subplot(mesh, fig, m1, [fesom[:,i]],
                                levels = levels,
                                units=unitphc, 
                                mapproj=self.mapproj, # robinson projection takes more time!
                                cmap = self.cmap,
                                cmap_extension=self.cmap_extension,
                                titles=labelfesom+'\n ({0}-{1} m m)'.format(uplow[0],uplow[1]),
                               )

                    m2 = axes['B']
                    f2 = pf.subplot(mesh, fig, m2, [temp_int_ma[:,i]], 
                                levels = levels,
                                units=unitphc, 
                                mapproj=self.mapproj, # robinson projection takes more time!
                                cmap = self.cmap,
                                cmap_extension=self.cmap_extension,
                                titles=labelphc+'\n ({0}-{1} m m)'.format(uplow[0],uplow[1]),
                               )

                    cbar1_ax = fig.add_axes([0.13, 0.53, 0.76, 0.02])
                    cbar1 = fig.colorbar(f1,
                                    cax = cbar1_ax, 
                                    orientation = 'horizontal',
                                    fraction=0.046, pad=0.04) 
                    cbar1.set_label(unitfesom, fontsize=18)
                    cbar1.ax.tick_params(labelsize=18)

                    m3 = axes['C']

                    levels_diff = np.arange(-5,5.25,0.25)
                    f3 = pf.subplot(mesh, fig, m3, [fesom[:,i]-temp_int_ma[:,i]], 
                                rowscol= (1,1),
                                levels = levels_diff,
                                units=unitphc, 
                                mapproj=self.mapproj, # robinson projection takes more time!
                                cmap = 'RdBu_r',
                                cmap_extension='both',
                                titles='FESOM - PHC'+' ({0}-{1} m)'.format(uplow[0],uplow[1]),
                               )

                    fig.subplots_adjust(bottom=0.02)
                    cbar2_ax = fig.add_axes([0.13, 0.001, 0.76, 0.02])
                    cbar2 = fig.colorbar(f3,
                                    cax = cbar2_ax, 
                                    orientation = 'horizontal',
                                    fraction=0.046, pad=0.04) 
                    cbar2.set_label(unitfesom, fontsize=18)
                    cbar2.ax.tick_params(labelsize=18)
                    
                    # fig export  -------------------------------------------------------------------------------------
                    if(self.savefig==True):
                        plt.savefig(self.savepath+self.runname+'_'+'Temp_PHC'+'_'+str(years[0])+'to'+str(years[-1])+'_{0}-{1}m.png'.format(uplow[0], uplow[1]), 
                                dpi = 300, bbox_inches='tight')
                        plt.savefig(self.savepath+self.runname+'_'+'Temp_PHC'+'_'+str(years[0])+'to'+str(years[-1])+'_{0}-{1}m.png'.format(uplow[0], uplow[1]), 
                                bbox_inches='tight')
                    plt.show(block=False) 
                
            if Taylor:
                # statistics  -------------------------------------------------------------------------------------
                # preparation of datasets
                if np.isnan(np.min(temp_int_ma)): print('WARNING: The interpolated PHC field contains NaNs at depth')
                if np.isnan(np.min(fesom)): print('WARNING: The interpolated FESOM field contains NaNs at depth')

                title = 'Normalized Taylor Diagram for Temperature'
                plt_Taylor_comp(temp_int_ma,fesom,mask=True,title=title, depth_array=depth_array, mesh=mesh,verbose = True)
                
                
                # fig export  -------------------------------------------------------------------------------------
                if(self.savefig==True):          
                    plt.savefig(self.savepath+self.runname+'_'+'Temp_PHC_Taylor'+'_'+str(years[0])+'_{0}-{1}m.png'.format(uplow[0], uplow[1]), 
                                dpi = 300, bbox_inches='tight')
                    plt.savefig(self.savepath+self.runname+'_'+'Temp_PHC_Taylor'+'_'+str(years[0])+'_{0}-{1}m.png'.format(uplow[0], uplow[1]), 
                                bbox_inches='tight')
                plt.show(block=False)  
                
            if output:
                    print('Only return non-layerwise output')
                
        # mean over depth  -------------------------------------------------------------------------------------
        else:
            if uplow[1] < -np.max(PHC_input.layer_depths):
                    print('{0}m is too deep for climatology.\nPlease consider choosing max depth at {1}!\n***'.format(uplow[1],np.max(GLODAP_input.layer_depths)))

            fesom_mean = pf.layermean_data(fesom, mesh, uplow = uplow)
            temp_int_ma_mean = pf.layermean_data(temp_int_ma, mesh, uplow = uplow)

            if(self.verbose):
                print('\nFESOM mean = {0:4.4f}, std = {1:4.4f}, min = {2:4.4f}, max = {3:4.4f}\nWOA mean = {4:4.4f}, std = {5:4.4f}, min = {6:4.4f}, max = {7:4.4f}'.format(
                np.nanmean(fesom_mean),np.nanstd(fesom_mean),np.nanmin(fesom_mean),np.nanmax(fesom_mean),
                np.nanmean(temp_int_ma_mean),np.nanstd(temp_int_ma_mean),np.nanmin(temp_int_ma_mean),np.nanmax(temp_int_ma_mean)))
            
        
            
            if plotting:
                fig = plt.figure(figsize=(15,12), constrained_layout=False)
                axes = fig.subplot_mosaic(
                            """
                            AB
                            CC
                            """,
                            gridspec_kw={'hspace': 0.1, 'wspace': 0.1, 'bottom': 0.03}, 
                            subplot_kw=dict(projection=self.mapproj))
                    
                m1 = axes['A']
                levels = np.arange(-2,25,1)
                f1 = pf.subplot(mesh, fig, m1, [fesom_mean],
                            levels = levels,
                            units=unitphc, 
                            mapproj=self.mapproj, # robinson projection takes more time!
                            cmap = self.cmap,
                            cmap_extension=self.cmap_extension,
                            titles=labelfesom+'\n ({0}-{1} m)'.format(uplow[0],uplow[1]),
                           )
                    
                m2 = axes['B']
                f2 = pf.subplot(mesh, fig, m2, [temp_int_ma_mean], 
                            levels = levels,
                            units=unitphc, 
                            mapproj=self.mapproj, # robinson projection takes more time!
                            cmap = self.cmap,
                            cmap_extension=self.cmap_extension,
                            titles=labelphc+'\n ({0}-{1} m)'.format(uplow[0],uplow[1]),
                           )
                    
                cbar1_ax = fig.add_axes([0.13, 0.53, 0.76, 0.02])
                cbar1 = fig.colorbar(f1,
                                cax = cbar1_ax, 
                                orientation = 'horizontal',
                                fraction=0.046, pad=0.04)
                cbar1.set_label(unitfesom, fontsize=18)
                cbar1.ax.tick_params(labelsize=18)
        
                m3 = axes['C']

                levels_diff = np.arange(-5,5.25,0.25)
                f3 = pf.subplot(mesh, fig, m3, [fesom_mean-temp_int_ma_mean], 
                            rowscol= (1,1),
                            levels = levels_diff,
                            units=unitphc, 
                            mapproj=self.mapproj, # robinson projection takes more time!
                            cmap = 'RdBu_r',
                            cmap_extension='both',
                            titles='FESOM - PHC'+' ({0}-{1} m)'.format(uplow[0],uplow[1]),
                           )
                
                fig.subplots_adjust(bottom=0.02)
                cbar2_ax = fig.add_axes([0.13, 0.001, 0.76, 0.02])
                cbar2 = fig.colorbar(f3,
                                cax = cbar2_ax, 
                                orientation = 'horizontal',
                                fraction=0.046, pad=0.04) 
                cbar2.set_label(unitfesom, fontsize=18)
                cbar2.ax.tick_params(labelsize=18)

                
                # fig export  -------------------------------------------------------------------------------------
                if(self.savefig==True):
                    plt.savefig(self.savepath+self.runname+'_'+'Temp_PHC'+'_'+str(years[0])+'to'+str(years[-1])+'_{0}-{1}m.png'.format(uplow[0], uplow[1]),
                            dpi = 300, bbox_inches='tight')
                    plt.savefig(self.savepath+self.runname+'_'+'Temp_PHC'+'_'+str(years[0])+'to'+str(years[-1])+'_{0}-{1}m.png'.format(uplow[0], uplow[1]),
                            bbox_inches='tight')
                plt.show(block=False) 
                
                if Taylor:
                    # statistics  -------------------------------------------------------------------------------------            
                    # preparation of datasets
                    if np.isnan(np.min(temp_int_ma_mean)): print('WARNING: The interpolated PHC field contains NaNs at depth')
                    if np.isnan(np.min(fesom_mean)): print('WARNING: The interpolated FESOM field contains NaNs at depth')

                    title = 'Taylor Diagram for T \n(mean over depth,  max = {0}-{1}m)'.format(uplow[0],uplow[1]),
                    plt_Taylor_norm(temp_int_ma_mean,fesom_mean,mask=True,title=title, verbose = True)
                    
                    
                    # fig export  -------------------------------------------------------------------------------------
                    if(self.savefig==True):
                        plt.savefig(self.savepath+self.runname+'_'+'Temp_PHC_Taylor'+'_'+str(years[0])+'to'+str(years[-1])+'.png', 
                                dpi = 300, bbox_inches='tight')
                        plt.savefig(self.savepath+self.runname+'_'+'Temp_PHC_Taylor'+'_'+str(years[0])+'to'+str(years[-1])+'.pdf', 
                                bbox_inches='tight')
                    plt.show(block=False)
                    
            if output:
                self.temp_fesom = fesom_mean
                self.temp_phc   = temp_int_ma_mean
        


In [4]:
class PHC3tempcomp_regulargrid:
    '''
    class PHC3tempcomp
    
    Compare PHC Temperature and Salinity to FESOM
    
    -use layerwise = True to compare a set of depth
    -use depth_limit to specify maximum depth for mean-over-depth comparison
    '''
    def __init__(self,runname,resultpath,savepath,mesh,ncpath,first_year,last_year,
                 mapproj='pc',
                 cmap = 'viridis',
                 savefig=False,
                 layerwise=False,depth_array=[],
                 depth_limit=100,
                 cmap_extension='max',
                 verbose=False):

        self.runname = runname
        self.resultpath = resultpath
        self.savepath = savepath
        self.mesh = mesh
        self.ncpath = ncpath
        self.fyear = first_year
        self.lyear = last_year
        self.mapproj = mapproj
        self.cmap = cmap
        self.savefig = savefig
        self.layerwise = layerwise
        self.depth_array = depth_array
        self.depth_limit = depth_limit
        self.verbose = verbose
        self.cmap_extension = cmap_extension

        import matplotlib.pyplot as plt
        import numpy as np
        from netCDF4 import Dataset
        from scipy.interpolate import griddata
        import skill_metrics as sm
        import cartopy.crs as ccrs
        import cartopy.feature as cfeature
        
        import pyfesom2 as pf

        from plot_Taylor_normalized import plt_Taylor_norm
        from plot_Taylor_normalized import plt_Taylor_comp

        # load FESOM mesh -------------------------------------------------------------------------------------
        #mesh       = pf.load_mesh(meshpath)
        years = np.arange(self.fyear, self.lyear+1,1)

        # check variables
        #NCfesom = self.resultpath + '/DFe.'+self.runname+'.'+str(self.fyear)+'.nc'
        #!ncdump -h $NCfesom

        labelfesom = 'FESOM ({0}-{1})'.format(self.fyear,self.lyear)
        unitfesom = 'T [$^{\circ}$C]'
        
        labelphc3 = 'PHC3'
        unitphc3 = 'T [$^{\circ}$C]'

        # load FESOM data -------------------------------------------------------------------------------------
        Tempfesom = pf.get_data(resultpath, "temp", years, mesh, 
                               how="mean", compute=True, runid=self.runname, silent=True)

        # load FESOM mesh diag -------------------------------------------------------------------------------
        meshdiag= self.resultpath+'/'+self.runname+'.mesh.diag.nc'
        #!ncdump -h $meshdiag

        diag = pf.get_meshdiag(mesh,meshdiag=meshdiag, runid=self.runname)             
        w = pf.climatology('/work/ollie/ogurses/input/phc3.0_annual.nc')
        
        box=[-180, 180, -89, 90]
        left, right, down, up = box
            
        levels = np.arange(-2,25,1)
        levels_diff = np.arange(-5,5.25,.25)

        # plot PHC3 and FESOM ----------------------------------------------------------------------------------
        if(self.layerwise):
            if(self.depth_array == []):
                depth_array = (0,50,200,1000,2000,4000)

            for d in depth_array:
                if d < np.nanmin(w.z):
                    print('{0}m is too deep for climatology.\nPlease consider choosing max depth at {1}!\n***'.format(d,np.max(PHC.layer_depths)))
    
                # get mesh index closest to desired depth
                ii = pf.ind_for_depth(d,mesh)
                iz, xx, yy, zz_temp = pf.fesom2clim(Tempfesom[:,ii], d, mesh, w, verbose=False)
                
                tfesom = zz_temp
                tphc3 = np.squeeze(w.T[iz,:,:]) 
            
                fig, axes = plt.subplots(ncols=2, nrows=1, constrained_layout=True,figsize=(15,15),
                                     subplot_kw=dict(projection=ccrs.PlateCarree()))

             
                axes[0].set_extent([left, right, down, up], crs=ccrs.PlateCarree())
                im0 = axes[0].contourf(xx, yy, tphc3, levels = levels,cmap=cmap, extend=cmap_extension, zlev=0,
                    transform=ccrs.PlateCarree());
                axes[0].add_feature(cfeature.LAND, zorder=1, edgecolor='none', facecolor='lightgray')
                axes[0].set_title(labelphc3, size = 20)

                axes[1].set_extent([left, right, down, up], crs=ccrs.PlateCarree())
                im1 = axes[1].contourf(xx, yy, tfesom, levels = levels,cmap=cmap, extend=cmap_extension, zlev=0,
                    transform=ccrs.PlateCarree());
                axes[1].add_feature(cfeature.LAND, zorder=1, edgecolor='none', facecolor='lightgray')
                axes[1].set_title(labelfesom, size = 20)

                cb = fig.colorbar(
                                im1, orientation="horizontal", ax=axes.ravel().tolist(), pad=0.01, shrink=0.9
                            )
                cb.ax.tick_params(labelsize=15)
                cb.set_label(unitfesom, size=20)


                fig, axes = plt.subplots(ncols=1, nrows=1, constrained_layout=True,figsize=(15,10),
                                         subplot_kw=dict(projection=ccrs.PlateCarree()))
                axes.set_extent([left, right, down, up], crs=ccrs.PlateCarree())
                im = axes.contourf(xx, yy, tphc3 - tfesom, levels = levels_diff,cmap='RdBu_r', extend='both', zlev=0,
                    transform=ccrs.PlateCarree());
                axes.add_feature(cfeature.LAND, zorder=1, edgecolor='none', facecolor='lightgray')
                axes.set_title('PHC3 - FESOM', size = 20)

                cb = fig.colorbar(
                                im, orientation="horizontal", ax=axes, pad=0.01, shrink=0.9
                            )
                cb.ax.tick_params(labelsize=15)
                cb.set_label(unitfesom, size=20)
       
            if(self.savefig==True): print('\n***\n***Too many figures to export...\n***')
        
        # mean over depth  -------------------------------------------------------------------------------------
        else:
            ii = pf.ind_for_depth(depth_limit,mesh)
            iz, xx, yy, zz_temp = pf.fesom2clim(np.nanmean(Tempfesom[:,:ii],axis=1), depth_limit, mesh, w, verbose=False)
            
            # mean over depth                       
            tfesom = zz_temp
            tphc3 = np.mean(w.T[:iz,:,:], axis = 0) 
            
            fig, axes = plt.subplots(ncols=2, nrows=1, constrained_layout=True,figsize=(15,15),
                                     subplot_kw=dict(projection=ccrs.PlateCarree()))

#            fig = plt.figure(constrained_layout=True,figsize=(15,15))
#             axd = fig.subplot_mosaic(
#                                 """
#                                 AB
#                                 CC
#                                 CC
#                                 """
#                             ,subplot_kw=dict(projection=ccrs.PlateCarree()))
             
            axes[0].set_extent([left, right, down, up], crs=ccrs.PlateCarree())
            im0 = axes[0].contourf(xx, yy, tphc3, levels = levels,cmap=cmap, extend=cmap_extension, zlev=0,
                transform=ccrs.PlateCarree());
            axes[0].add_feature(cfeature.LAND, zorder=1, edgecolor='none', facecolor='lightgray')
            axes[0].set_title(labelphc3, size = 20)
            
            axes[1].set_extent([left, right, down, up], crs=ccrs.PlateCarree())
            im1 = axes[1].contourf(xx, yy, tfesom, levels = levels,cmap=cmap, extend=cmap_extension, zlev=0,
                transform=ccrs.PlateCarree());
            axes[1].add_feature(cfeature.LAND, zorder=1, edgecolor='none', facecolor='lightgray')
            axes[1].set_title(labelfesom, size = 20)
        
            cb = fig.colorbar(
                            im1, orientation="horizontal", ax=axes.ravel().tolist(), pad=0.01, shrink=0.9
                        )
            cb.ax.tick_params(labelsize=15)
            cb.set_label(unitfesom, size=20)
            
            
            fig, axes = plt.subplots(ncols=1, nrows=1, constrained_layout=True,figsize=(15,10),
                                     subplot_kw=dict(projection=ccrs.PlateCarree()))
            axes.set_extent([left, right, down, up], crs=ccrs.PlateCarree())
            im = axes.contourf(xx, yy, tphc3 - tfesom, levels = levels_diff,cmap='RdBu_r', extend='both', zlev=0,
                transform=ccrs.PlateCarree());
            axes.add_feature(cfeature.LAND, zorder=1, edgecolor='none', facecolor='lightgray')
            axes.set_title('PHC3 - FESOM', size = 20)
            
            cb = fig.colorbar(
                            im, orientation="horizontal", ax=axes, pad=0.01, shrink=0.9
                        )
            cb.ax.tick_params(labelsize=15)
            cb.set_label(unitfesom, size=20)
            
            plt.show(block=False) 
            # fig export  -------------------------------------------------------------------------------------
            if(self.savefig==True):
                plt.savefig(self.savepath+self.runname+'_'+'Temp_PHC3'+'_'+str(years[0])+'to'+str(years[-1])+'.png', 
                        dpi = 300, bbox_inches='tight') 

In [5]:
class PHC3salcomp_regulargrid:
    '''
    class PHC3tempcomp
    
    Compare PHC Temperature and Salinity to FESOM
    
    -use layerwise = True to compare a set of depth
    -use depth_limit to specify maximum depth for mean-over-depth comparison
    '''
    def __init__(self,runname,resultpath,savepath,mesh,ncpath,first_year,last_year,
                 mapproj='pc',
                 cmap = 'viridis',
                 savefig=False,
                 layerwise=False,depth_array=[],
                 depth_limit=100,
                 verbose=False):

        self.runname = runname
        self.resultpath = resultpath
        self.savepath = savepath
        self.mesh = mesh
        self.ncpath = ncpath
        self.fyear = first_year
        self.lyear = last_year
        self.mapproj = mapproj
        self.cmap = cmap
        self.savefig = savefig
        self.layerwise = layerwise
        self.depth_array = depth_array
        self.depth_limit = depth_limit
        self.verbose = verbose

        import matplotlib.pyplot as plt
        import numpy as np
        from netCDF4 import Dataset
        from scipy.interpolate import griddata
        import skill_metrics as sm
        import cartopy.crs as ccrs
        import cartopy.feature as cfeature
        
        import pyfesom2 as pf

        from plot_Taylor_normalized import plt_Taylor_norm
        from plot_Taylor_normalized import plt_Taylor_comp

        # load FESOM mesh -------------------------------------------------------------------------------------
        #mesh       = pf.load_mesh(meshpath)
        years = np.arange(self.fyear, self.lyear+1,1)

        # check variables
        #NCfesom = self.resultpath + '/DFe.'+self.runname+'.'+str(self.fyear)+'.nc'
        #!ncdump -h $NCfesom

        labelfesom = 'FESOM ({0}-{1})'.format(self.fyear,self.lyear)
        unitfesom = ''
        
        labelphc3 = 'PHC3'
        unitphc3 = ''

        # load FESOM data -------------------------------------------------------------------------------------
        Tempfesom = pf.get_data(resultpath, "salt", years, mesh, 
                               how="mean", compute=True, runid=self.runname, silent=True)

        # load FESOM mesh diag -------------------------------------------------------------------------------
        meshdiag= self.resultpath+'/'+self.runname+'.mesh.diag.nc'
        #!ncdump -h $meshdiag

        diag = pf.get_meshdiag(mesh,meshdiag=meshdiag, runid=self.runname)             
        w = pf.climatology('/work/ollie/ogurses/input/phc3.0_annual.nc')
        
        levels = np.arange(23,37,.5)
        levels_diff = np.arange(-3,3.2,0.2)
        
        box=[-180, 180, -89, 90]
        left, right, down, up = box
        
        # plot PHC3 and FESOM ----------------------------------------------------------------------------------
        if(self.layerwise):
            if(self.depth_array == []):
                depth_array = (0,50,200,1000,2000,4000)

            for d in depth_array:
                if d < np.min(w.z):
                    print('{0}m is too deep for climatology.\nPlease consider choosing max depth at {1}!\n***'.format(d,np.max(PHC.layer_depths)))
    
                # get mesh index closest to desired depth
                ii = pf.ind_for_depth(d,mesh)
                iz, xx, yy, zz_temp = pf.fesom2clim(Tempfesom[:,ii], d, mesh, w, verbose=False)
                
                tfesom = zz_temp
                tphc3 = np.squeeze(w.S[iz,:,:]) 
            
                fig, axes = plt.subplots(ncols=2, nrows=1, constrained_layout=True,figsize=(15,15),
                                     subplot_kw=dict(projection=ccrs.PlateCarree()))

             
                axes[0].set_extent([left, right, down, up], crs=ccrs.PlateCarree())
                im0 = axes[0].contourf(xx, yy, tphc3, levels = levels,cmap=cmap, extend='both', zlev=0,
                    transform=ccrs.PlateCarree());
                axes[0].add_feature(cfeature.LAND, zorder=1, edgecolor='none', facecolor='lightgray')
                axes[0].set_title(labelphc3, size = 20)

                axes[1].set_extent([left, right, down, up], crs=ccrs.PlateCarree())
                im1 = axes[1].contourf(xx, yy, tfesom, levels = levels,cmap=cmap, extend='both', zlev=0,
                    transform=ccrs.PlateCarree());
                axes[1].add_feature(cfeature.LAND, zorder=1, edgecolor='none', facecolor='lightgray')
                axes[1].set_title(labelfesom, size = 20)

                cb = fig.colorbar(
                                im1, orientation="horizontal", ax=axes.ravel().tolist(), pad=0.01, shrink=0.9
                            )
                cb.ax.tick_params(labelsize=15)
                cb.set_label(unitfesom, size=20)


                fig, axes = plt.subplots(ncols=1, nrows=1, constrained_layout=True,figsize=(15,10),
                                         subplot_kw=dict(projection=ccrs.PlateCarree()))
                axes.set_extent([left, right, down, up], crs=ccrs.PlateCarree())
                im = axes.contourf(xx, yy, tphc3 - tfesom, levels = levels_diff,cmap='RdBu_r', extend='both', zlev=0,
                    transform=ccrs.PlateCarree());
                axes.add_feature(cfeature.LAND, zorder=1, edgecolor='none', facecolor='lightgray')
                axes.set_title('PHC3 - FESOM', size = 20)

                cb = fig.colorbar(
                                im, orientation="horizontal", ax=axes, pad=0.01, shrink=0.9
                            )
                cb.ax.tick_params(labelsize=15)
                cb.set_label(unitfesom, size=20)

       
            if(self.savefig==True): print('\n***\n***Too many figures to export...\n***')
        
        # mean over depth  -------------------------------------------------------------------------------------
        else:
            ii = pf.ind_for_depth(depth_limit,mesh)
            iz, xx, yy, zz_temp = pf.fesom2clim(np.nanmean(Tempfesom[:,:ii],axis=1), depth_limit, mesh, w, verbose=False)
            
            # mean over depth                       
            tfesom = zz_temp
            tphc3 = np.mean(w.S[:iz,:,:], axis = 0) 
            
            fig, axes = plt.subplots(ncols=2, nrows=1, constrained_layout=True,figsize=(15,15),
                                     subplot_kw=dict(projection=ccrs.PlateCarree()))

             
            axes[0].set_extent([left, right, down, up], crs=ccrs.PlateCarree())
            im0 = axes[0].contourf(xx, yy, tphc3, levels = levels,cmap=cmap, extend='both', zlev=0,
                transform=ccrs.PlateCarree());
            axes[0].add_feature(cfeature.LAND, zorder=1, edgecolor='none', facecolor='lightgray')
            axes[0].set_title(labelphc3, size = 20)
            
            axes[1].set_extent([left, right, down, up], crs=ccrs.PlateCarree())
            im1 = axes[1].contourf(xx, yy, tfesom, levels = levels,cmap=cmap, extend='both', zlev=0,
                transform=ccrs.PlateCarree());
            axes[1].add_feature(cfeature.LAND, zorder=1, edgecolor='none', facecolor='lightgray')
            axes[1].set_title(labelfesom, size = 20)
        
            cb = fig.colorbar(
                            im1, orientation="horizontal", ax=axes.ravel().tolist(), pad=0.01, shrink=0.9
                        )
            cb.ax.tick_params(labelsize=15)
            cb.set_label(unitfesom, size=20)
            
            
            fig, axes = plt.subplots(ncols=1, nrows=1, constrained_layout=True,figsize=(15,10),
                                     subplot_kw=dict(projection=ccrs.PlateCarree()))
            axes.set_extent([left, right, down, up], crs=ccrs.PlateCarree())
            im = axes.contourf(xx, yy, tphc3 - tfesom, levels = levels_diff,cmap='RdBu_r', extend='both', zlev=0,
                transform=ccrs.PlateCarree());
            axes.add_feature(cfeature.LAND, zorder=1, edgecolor='none', facecolor='lightgray')
            axes.set_title('PHC3 - FESOM', size = 20)
            
            cb = fig.colorbar(
                            im, orientation="horizontal", ax=axes, pad=0.01, shrink=0.9
                        )
            cb.ax.tick_params(labelsize=15)
            cb.set_label(unitfesom, size=20)
            
            plt.show(block=False) 
            # fig export  -------------------------------------------------------------------------------------
            if(self.savefig==True):
                plt.savefig(self.savepath+self.runname+'_'+'Temp_PHC3'+'_'+str(years[0])+'to'+str(years[-1])+'.png', 
                        dpi = 300, bbox_inches='tight') 